### Import Required Libraries and Set Up Environment Variables

First, we'll import the necessary libraries and load our NASA API key.

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime

# Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

# Verify API key is loaded
if NASA_API_KEY is None:
    raise ValueError("NASA_API_KEY not found in .env file")
print("API key loaded successfully")

API key loaded successfully


### Part 1: Request CME data from the NASA API

First, we'll fetch and process the CME (Coronal Mass Ejection) data.

In [2]:
# Set the base URL to NASA's DONKI API
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs
specifier = "CME"

# Search for CMEs between a begin and end date
startDate = "2013-05-01"
endDate = "2024-05-01"

# Build URL for CME
query_url_CME = f"{base_url}{specifier}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

# Make a GET request and store response
cme_response = requests.get(query_url_CME)
cme_json = cme_response.json()

# Preview first result
print("Sample CME Data:")
print(json.dumps(cme_json[0], indent=4))

Sample CME Data:
{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechni

In [3]:
# Convert to DataFrame
cme_df = pd.DataFrame(cme_json)

# Keep only required columns
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

# Remove rows with missing linkedEvents
cme_df = cme_df.dropna(subset=['linkedEvents'])

# Initialize empty list for expanded rows
expanded_rows = []

# Iterate through each row
for i in cme_df.index:
    activityID = cme_df.loc[i, 'activityID']
    startTime = cme_df.loc[i, 'startTime']
    linkedEvents = cme_df.loc[i, 'linkedEvents']
    
    # Iterate through linked events
    for event in linkedEvents:
        expanded_rows.append({
            'activityID': activityID,
            'startTime': startTime,
            'linkedEvent': event
        })

# Create DataFrame from expanded rows
cme_expanded_df = pd.DataFrame(expanded_rows)

# Display the first few rows
print("\nExpanded CME DataFrame:")
print(cme_expanded_df.head())


Expanded CME DataFrame:
                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                     linkedEvent  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  


In [4]:
# Function to extract activityID from dict
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict['activityID']
    except (ValueError, TypeError) as e:
        return None

# Test the function
print("\nTesting extract_activityID_from_dict function:")
print(extract_activityID_from_dict(cme_df.loc[0, 'linkedEvents'][0]))

# Apply function to create GST_ActivityID column
cme_expanded_df.loc[:, 'GST_ActivityID'] = cme_expanded_df['linkedEvent'].apply(extract_activityID_from_dict)

# Remove rows with missing GST_ActivityID
cme_expanded_df = cme_expanded_df.dropna(subset=['GST_ActivityID'])

# Convert columns to appropriate types
cme_expanded_df['GST_ActivityID'] = cme_expanded_df['GST_ActivityID'].astype(str)
cme_expanded_df['startTime'] = pd.to_datetime(cme_expanded_df['startTime'])

# Rename columns
cme_expanded_df = cme_expanded_df.rename(columns={
    'startTime': 'startTime_CME',
    'activityID': 'cmeID'
})

# Drop linkedEvent column
cme_expanded_df = cme_expanded_df.drop('linkedEvent', axis=1)

# Filter for GST events
cme_final_df = cme_expanded_df[cme_expanded_df['GST_ActivityID'].str.contains('GST')]

print("\nFinal CME DataFrame:")
print(cme_final_df.info())
print("\nFirst few rows:")
print(cme_final_df.head())


Testing extract_activityID_from_dict function:
2013-05-04T04:52:00-IPS-001

Final CME DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 21 to 1689
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           61 non-null     object             
 1   startTime_CME   61 non-null     datetime64[ns, UTC]
 2   GST_ActivityID  61 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.9+ KB
None

First few rows:
                           cmeID             startTime_CME  \
21   2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00   
48   2013-09-29T22:40:00-CME-001 2013-09-29 22:40:00+00:00   
90   2013-12-04T23:12:00-CME-001 2013-12-04 23:12:00+00:00   
148  2014-02-16T14:15:00-CME-001 2014-02-16 14:15:00+00:00   
151  2014-02-18T01:25:00-CME-001 2014-02-18 01:25:00+00:00   

                  GST_ActivityID  
21   2013-06-07T03:00:0

### Part 2: Request GST data from the NASA API

Now we'll fetch and process the GST (Geomagnetic Storm) data.

In [5]:
# Set the specifier for GSTs
specifier = "GST"

# Build URL for GST
query_url_GST = f"{base_url}{specifier}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

# Make a GET request and store response
gst_response = requests.get(query_url_GST)
gst_json = gst_response.json()

# Preview first result
print("Sample GST Data:")
print(json.dumps(gst_json[0], indent=4))

Sample GST Data:
{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [6]:
# Convert to DataFrame
gst_df = pd.DataFrame(gst_json)

# Keep only required columns
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]

# Remove rows with missing linkedEvents
gst_df = gst_df.dropna(subset=['linkedEvents'])

# Explode linkedEvents to create separate rows
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)

# Display the first few rows after explode
print("\nGST DataFrame after explode:")
print(gst_df.head())


GST DataFrame after explode:
                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  


In [7]:
# Apply extract_activityID_from_dict function
gst_df.loc[:, 'CME_ActivityID'] = gst_df['linkedEvents'].apply(extract_activityID_from_dict)

# Remove rows with missing CME_ActivityID
gst_df = gst_df.dropna(subset=['CME_ActivityID'])

# Convert columns to appropriate types
gst_df['gstID'] = gst_df['gstID'].astype(str)
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])

# Rename startTime
gst_df = gst_df.rename(columns={'startTime': 'startTime_GST'})

# Drop linkedEvents
gst_df = gst_df.drop('linkedEvents', axis=1)

# Filter for CME events
gst_final_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME')]

print("\nFinal GST DataFrame:")
print(gst_final_df.info())
print("\nFirst few rows:")
print(gst_final_df.head())


Final GST DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 1 to 203
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     object             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 1.9+ KB
None

First few rows:
                         gstID             startTime_GST  \
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
5  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
7  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
9  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID  
1  2013-06-02T20:24:00-CME-001  
2  2013-09-29T22:40:00-CME-001  
5  2013-12-04T23:12:00-CME-001  
7  2014-02-16T14

### Part 3: Merge and Clean the Data for Export

Finally, we'll merge the CME and GST data and calculate the time differences.

In [8]:
# Merge CME and GST DataFrames
merged_df = pd.merge(
    gst_final_df,
    cme_final_df,
    left_on=['gstID', 'CME_ActivityID'],
    right_on=['GST_ActivityID', 'cmeID']
)

# Verify row counts
print("Row counts:")
print(f"CME DataFrame: {len(cme_final_df)}")
print(f"GST DataFrame: {len(gst_final_df)}")
print(f"Merged DataFrame: {len(merged_df)}")

# Calculate time difference in hours
merged_df['timeDiff'] = (merged_df['startTime_GST'] - merged_df['startTime_CME']).dt.total_seconds() / 3600

# Show time difference statistics
print("\nTime Difference Statistics (hours):")
print(merged_df['timeDiff'].describe())

# Display first few rows of final dataset
print("\nFirst few rows of merged data:")
print(merged_df.head())

# Export to CSV
output_path = "6-output/collected_data.csv"
merged_df.to_csv(output_path, index=False)
print(f"\nData exported to: {output_path}")

Row counts:
CME DataFrame: 61
GST DataFrame: 61
Merged DataFrame: 61

Time Difference Statistics (hours):
count     61.000000
mean      71.490710
std       23.885927
min       32.600000
25%       52.000000
50%       67.850000
75%       85.583333
max      147.000000
Name: timeDiff, dtype: float64

First few rows of merged data:
                         gstID             startTime_GST  \
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-00